In [1]:
import pyaudio
import wave


def record_audio(RECORD_SECONDS, WAVE_OUTPUT_FILENAME):
    #--------- SETTING PARAMS FOR OUR AUDIO FILE ------------#
    FORMAT = pyaudio.paInt16    # format of wave
    CHANNELS = 2                # no. of audio channels
    RATE = 44100                # frame rate
    CHUNK = 1024                # frames per audio sample
    #--------------------------------------------------------#
     
    # creating PyAudio object
    audio = pyaudio.PyAudio()
     
    # open a new stream for microphone
    # It creates a PortAudio Stream Wrapper class object
    stream = audio.open(format=FORMAT,channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)


    #----------------- start of recording -------------------#
    print("Listening...")

    # list to save all audio frames
    frames = []

    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        # read audio stream from microphone
        data = stream.read(CHUNK)
        # append audio data to frames list
        frames.append(data)

    #------------------ end of recording --------------------#   
    print("Finished recording.")
      
    stream.stop_stream()    # stop the stream object
    stream.close()          # close the stream object
    audio.terminate()       # terminate PortAudio

    #------------------ saving audio ------------------------#

    # create wave file object
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')

    # settings for wave file object
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))

    # closing the wave file object
    waveFile.close()


def read_audio(WAVE_FILENAME):
    # function to read audio(wav) file
    with open(WAVE_FILENAME, 'rb') as f:
        audio = f.read()
    return audio

In [5]:
import requests
import json
#from Recorder import record_audio, read_audio

# Wit speech API endpoint
API_ENDPOINT = 'https://api.wit.ai/speech'

# Wit.ai api access token
wit_access_token = 'IYOTYHJ3LR5PUR3VUXT6BNJYUHGKQEGG'


def RecognizeSpeech(AUDIO_FILENAME, num_seconds =5):
    
    # record audio of specified length in specified audio file
    record_audio(num_seconds, AUDIO_FILENAME)
    
    # reading audio
    audio = read_audio(AUDIO_FILENAME)
    
    # defining headers for HTTP request
    headers = {'authorization': 'Bearer ' + wit_access_token,
               'Content-Type': 'audio/wav'}

    # making an HTTP post request
    resp = requests.post(API_ENDPOINT, headers = headers,
                         data = audio)
    
    # converting response content to JSON format
    data = json.loads(resp.content)
    
    # get text from data
    text = data['_text']
    
    # return the text
    return text


if __name__ == "__main__":
    text =  RecognizeSpeech('myspeech.wav', 4)
    print("\nYou said: {}".format(text))
    
    

Listening...
Finished recording.

You said: 


In [13]:
from wit import Wit
access_token= "IYOTYHJ3LR5PUR3VUXT6BNJYUHGKQEGG"
client= Wit(access_token=access_token)
message_text= text
resp=client.message(message_text)
print(resp)

{'_text': "i'm from china", 'entities': {'location': [{'confidence': 0.93733, 'value': 'china', 'resolved': {'values': [{'name': "People's Republic of China", 'grain': 'country', 'type': 'resolved', 'coords': {'lat': 35, 'long': 105}, 'external': {'geonames': '1814991', 'wikidata': 'Q148', 'wikipedia': 'China'}}]}}]}, 'msg_id': '11uV4zgCNjMA5LeU1'}


In [14]:
from wit import Wit 

wit_access_token = "IYOTYHJ3LR5PUR3VUXT6BNJYUHGKQEGG"
client = Wit(access_token = wit_access_token)

def wit_response(message_text):
	resp = client.message(message_text)
	entity = None
	value = None

	try:
		entity = list(resp['entities'])[0]
		value = resp['entities'][entity][0]['value']
	except:
		pass
        
	return (entity, value)

In [15]:
print(wit_response(text))

('location', 'china')


In [16]:
entity, value = wit_response(message_text)
if entity == 'newstype':
    response = "Ok, I will send you the {} news".format(str(value))
elif entity == 'location':
    response = "Ok, so you live in {0}. Here are top headlines from {0}".format(str(value))

if entity == None:
    response = "I have no idea what you are saying!"

In [17]:
print(response)

Ok, so you live in china. Here are top headlines from china


In [15]:
# Import the required module for text  
# to speech conversion 
from gtts import gTTS 
  
# This module is imported so that we can  
# play the converted audio 
import os 
  
# The text that you want to convert to audio 
mytext = response
  
# Language in which you want to convert 
language = 'en'
  
# Passing the text and language to the engine,  
# here we have marked slow=False. Which tells  
# the module that the converted audio should  
# have a high speed 
myobj = gTTS(text=mytext, lang=language, slow=False) 
  
# Saving the converted audio in a mp3 file named 
# welcome  
myobj.save("welcome.mp3") 
  
# Playing the converted file 
os.system("welcome.mp3") 

0